In [3]:
import tensorflow as tf, numpy as np, matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from IPython.display import HTML

# Import HTML safely (only used if running in notebook)
try:
    from IPython.display import HTML
    in_notebook = True
except ImportError:
    in_notebook = False

iris = load_iris()
X = iris.data[:, :2]  # Use sepal length and width only
X = StandardScaler().fit_transform(X)

X_tensor = tf.constant(X, dtype=tf.float32)
num_points, num_features = X_tensor.shape
k = 3  # number of clusters

tf.random.set_seed(42)
indices = tf.random.shuffle(tf.range(num_points))[:k]
centroids = tf.gather(X_tensor, indices)

centroid_history = [centroids.numpy()]
assignments_history = []
max_steps = 20

# ---- Choose your distance metric ----
# Options: "euclidean", "manhattan", "cosine", "sqeuclidean"
distance_metric = "cosine"   # 👈 change here to test different metrics

for step in range(max_steps):
    
    # Expand tensors for broadcasting
    expanded_points = tf.expand_dims(X_tensor, 1)      # (N, 1, D)
    expanded_centroids = tf.expand_dims(centroids, 0)  # (1, K, D)
    
    # ---- Compute distances based on selected metric ----
    if distance_metric == "euclidean":
        distances = tf.sqrt(tf.reduce_sum(tf.square(expanded_points - expanded_centroids), axis=2))
    elif distance_metric == "sqeuclidean":
        distances = tf.reduce_sum(tf.square(expanded_points - expanded_centroids), axis=2)
    elif distance_metric == "manhattan":
        distances = tf.reduce_sum(tf.abs(expanded_points - expanded_centroids), axis=2)
    elif distance_metric == "cosine":
        a = tf.nn.l2_normalize(expanded_points, axis=2)
        b = tf.nn.l2_normalize(expanded_centroids, axis=2)
        cosine_sim = tf.reduce_sum(a * b, axis=2)
        distances = 1 - cosine_sim  # convert similarity → distance
    else:
        raise ValueError("Unsupported metric! Use euclidean, sqeuclidean, manhattan, or cosine.")

    # Assign each point to nearest centroid
    cluster_assignments = tf.argmin(distances, axis=1)

    # Compute new centroids
    new_centroids = []
    for i in range(k):
        points_for_cluster = tf.boolean_mask(X_tensor, tf.equal(cluster_assignments, i))
        mean = tf.reduce_mean(points_for_cluster, axis=0)
        new_centroids.append(mean)
    new_centroids = tf.stack(new_centroids)

    # Record for animation
    assignments_history.append(cluster_assignments.numpy())
    centroid_history.append(new_centroids.numpy())

    # Check for convergence
    if tf.reduce_all(tf.equal(new_centroids, centroids)):
        print(f"Converged at step {step + 1} using {distance_metric} distance")
        break

    # Update centroids
    centroids = new_centroids


# ---- Compute Objective Function (J) for each iteration ----

objective_history = []
for frame in range(len(assignments_history)):
    assignments = assignments_history[frame]
    centroids = centroid_history[frame + 1]
    J = np.sum([
        np.sum((X[assignments == i] - centroids[i]) ** 2)
        for i in range(k)
    ])
    objective_history.append(J)

# ---- Create figure with 2 panels ----

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(11, 5))
cmap = plt.cm.get_cmap('viridis', k)

# --- Left: K-Means Clustering ---

init_assign = assignments_history[0]
init_centroids = centroid_history[1]
sc = ax1.scatter(X[:, 0], X[:, 1], c=init_assign, cmap=cmap, s=40, vmin=0, vmax=k-1)
centroid_ln, = ax1.plot(init_centroids[:, 0], init_centroids[:, 1],
                        'rx', markersize=12, markeredgewidth=2)
iteration_text = ax1.text(0.02, 0.95, "", transform=ax1.transAxes,
                          fontsize=12, color='black', weight='bold')

ax1.set_xlim(X[:, 0].min() - 0.5, X[:, 0].max() + 0.5)
ax1.set_ylim(X[:, 1].min() - 0.5, X[:, 1].max() + 0.5)
ax1.set_xlabel("Sepal Length (scaled)")
ax1.set_ylabel("Sepal Width (scaled)")
ax1.set_title("K-Means Clustering Process")

# --- Right: Objective Function Convergence ---

ax2.set_xlim(0, len(objective_history))
ax2.set_ylim(0, max(objective_history) * 1.05)
ax2.set_xlabel("Iteration")
ax2.set_ylabel("Objective Function J(K)")
ax2.set_title("Objective Function Convergence")
line_loss, = ax2.plot([], [], 'b-o')

# Objective value displayed just below title

text_loss = ax2.text(0.02, 0.93, "", transform=ax2.transAxes,
                     fontsize=11, color='darkred', va='top')

# ---- Update Function ----

def update(frame):
    assignments = assignments_history[frame]
    centroids = centroid_history[frame + 1]
    J = objective_history[frame]

    # Update clustering
    sc.set_array(np.array(assignments, dtype=float))
    centroid_ln.set_data(centroids[:, 0], centroids[:, 1])
    iteration_text.set_text(f"Iteration: {frame + 1}")

    # Update objective curve and title text
    line_loss.set_data(np.arange(frame + 1), objective_history[:frame + 1])
    ax2.set_title("Objective Function Convergence")
    text_loss.set_text(f"Objective Value (J): {J:.2f}")

    return sc, centroid_ln, line_loss, text_loss, iteration_text

# ---- Create and Display Animation ----

anim = FuncAnimation(
    fig, update,
    frames=len(assignments_history),
    interval=800,
    blit=True,
    repeat=False
)

plt.close(fig)  # prevent redundant static figure
HTML(anim.to_jshtml())  # show animation with buttons

Converged at step 12 using cosine distance


/var/folders/sl/t1mq9bsd6cx92whmzdxkddfw0000gn/T/ipykernel_94281/1839192705.py:94: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('viridis', k)
